**Импортим нужные библиотеки**

In [2]:
!pip install xmltodict

In [150]:
import xmltodict
import re

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

**Определяем функции с парсингом входных данных**

In [151]:
companies = {}
def get_sample_text(sample):
    assert sample['column'][3]['@name'] == 'text'
    return sample['column'][3]['#text']


def get_sample_answers_bank(sample):
    answers = {}
    for i in range(4, 12):
        companies[sample['column'][i]['@name']] = i
        answers[sample['column'][i]['@name']] = None if sample['column'][i]['#text'] == 'NULL'\
            else int(sample['column'][i]['#text'])
    return answers

def get_sample_answers_tkk(sample):
    answers = {}
    for i in range(4, 11):
        companies[sample['column'][i]['@name']] = i
        answers[sample['column'][i]['@name']] = None if sample['column'][i]['#text'] == 'NULL'\
            else int(sample['column'][i]['#text'])
    return answers

def get_sample_id(sample):
    assert sample['column'][0]['@name'] == 'id'
    return int(sample['column'][0]['#text'])


def get_data(filename):
    df = pd.DataFrame()
    with open(filename, "r", encoding='utf-8') as f:
        d = xmltodict.parse(f.read(), process_namespaces=True)
        clean_samples = []
        for sample in d['pma_xml_export']['database']['table']:
            sample_id = get_sample_id(sample)
            text = get_sample_text(sample)
            answers = get_sample_answers_bank(sample)
            for company, answer in answers.items():
                if answer is not None:
                    clean_samples.append((sample_id, text, company, answer))
        df['text'] = [sample[1] for sample in clean_samples]
        df['answer'] = [sample[3] for sample in clean_samples]
        df['company'] = [sample[2] for sample in clean_samples]
        df['sample_id'] = [sample[0] for sample in clean_samples]
    return df

def get_data_tkk(filename):
    df = pd.DataFrame()
    with open(filename, "r", encoding='utf-8') as f:
        d = xmltodict.parse(f.read(), process_namespaces=True)
        clean_samples = []
        for sample in d['pma_xml_export']['database']['table']:
            sample_id = get_sample_id(sample)
            text = get_sample_text(sample)
            answers = get_sample_answers_tkk(sample)
            for company, answer in answers.items():
                if answer is not None:
                    clean_samples.append((sample_id, text, company, answer))
        df['text'] = [sample[1] for sample in clean_samples]
        df['answer'] = [sample[3] for sample in clean_samples]
        df['company'] = [sample[2] for sample in clean_samples]
        df['sample_id'] = [sample[0] for sample in clean_samples]
    return df

In [152]:
train_filename = "/content/drive/My Drive/Colab Notebooks/bank_train_2016.xml"
test_filename = "/content/drive/My Drive/Colab Notebooks/banks_test_etalon.xml"

train = get_data(train_filename)
test = get_data(test_filename)

**Заменяем все ссылки и юзернеймы на url и user соответственно**

In [154]:
url_replacement = lambda x: re.sub(r'(?:http[^\s]+)($|\s)', r'url\1', x)
user_replacement = lambda x: re.sub(r'(?:@[^\s]+)($|\s)', r'user\1', x)

train['text'] = train['text'].apply(url_replacement)
train['text'] = train['text'].apply(user_replacement)

test['text'] = test['text'].apply(url_replacement)
test['text'] = test['text'].apply(user_replacement)

**Преобразование нашей обучающей выборки по tf-idf**

In [155]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train['text'])
X_train.shape

(10725, 11703)

**Преобразование нашей тестовой выборки**

In [156]:
X_test = vectorizer.transform(test['text'])
X_test.shape

(3418, 11703)

**Обучение логистической регрессии**

In [157]:
lr = LogisticRegression(solver='lbfgs', multi_class='multinomial', class_weight='balanced')
lr.fit(X_train, train['answer'])
y_pred = lr.predict(X_test)

**Смотрим на результаты**

In [158]:
f1_score(test['answer'], y_pred, average='micro', labels=[-1,1])

0.49823460180462925

In [159]:
f1_score(test['answer'], y_pred, average='macro', labels=[-1,1])

0.4698891720341647

**Запустить этот же ноутбук на данных ttk**

In [160]:
train_filename = "/content/drive/My Drive/Colab Notebooks/tkk_train_2016.xml"
test_filename = "/content/drive/My Drive/Colab Notebooks/tkk_test_etalon.xml"

train_tkk = get_data_tkk(train_filename)
test_tkk = get_data_tkk(test_filename)

In [161]:
url_replacement = lambda x: re.sub(r'(?:http[^\s]+)($|\s)', r'url\1', x)
user_replacement = lambda x: re.sub(r'(?:@[^\s]+)($|\s)', r'user\1', x)

train_tkk['text'] = train_tkk['text'].apply(url_replacement)
train_tkk['text'] = train_tkk['text'].apply(user_replacement)

test_tkk['text'] = test_tkk['text'].apply(url_replacement)
test_tkk['text'] = test_tkk['text'].apply(user_replacement)

In [162]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_tkk['text'])
X_train.shape

(9209, 16056)

In [163]:
X_test = vectorizer.transform(test_tkk['text'])
X_test.shape

(2460, 16056)

In [164]:
lr = LogisticRegression(solver='lbfgs', multi_class='multinomial', class_weight='balanced')
lr.fit(X_train, train_tkk['answer'])
y_pred = lr.predict(X_test)

In [89]:
f1_score(test_tkk['answer'], y_pred, average='micro', labels=[-1,1])

0.6103756708407871

In [165]:
f1_score(test_tkk['answer'], y_pred, average='macro', labels=[-1,1])

0.4835264679302881

**Сверточная нейронная сеть для анализа твитов**

Нормализация текста

In [167]:
import string
#для ттк данных

train_tkk['text'] = train_tkk['text'].str.lower()
train_tkk['text'] = train_tkk['text'].str.strip(" ")
train_tkk['text'] = train_tkk['text'].str.replace('\s+', " ")

test_tkk['text'] = test_tkk['text'].str.lower()
test_tkk['text'] = test_tkk['text'].str.strip(" ")
test_tkk['text'] = test_tkk['text'].str.replace('\s+', " ")

for c in string.punctuation:
    train_tkk['text'] = train_tkk['text'].str.replace(c, "")
for c in string.punctuation:
    test_tkk['text'] = test_tkk['text'].str.replace(c, "")


In [168]:
#для банк данных
train['text'] = train['text'].str.lower()
train['text'] = train['text'].str.strip(" ")
train['text'] = train['text'].str.replace('\s+', " ")

test['text'] = test['text'].str.lower()
test['text'] = test['text'].str.strip(" ")
test['text'] = test['text'].str.replace('\s+', " ")

for c in string.punctuation:
    train['text'] = train['text'].str.replace(c, "")
for c in string.punctuation:
    test['text'] = test['text'].str.replace(c, "")

In [169]:
#ттк
train_tkk['text'] = train_tkk['text'].str.replace('’', "")
train_tkk['text'] = train_tkk['text'].str.replace('”', "")
train_tkk['text'] = train_tkk['text'].str.replace('“', "")
train_tkk['text'] = train_tkk['text'].str.replace('—', "")
train_tkk['text'] = train_tkk['text'].str.replace('«', "")
train_tkk['text'] = train_tkk['text'].str.replace('»', "")

test_tkk['text'] = test_tkk['text'].str.replace('’', "")
test_tkk['text'] = test_tkk['text'].str.replace('”', "")
test_tkk['text'] = test_tkk['text'].str.replace('“', "")
test_tkk['text'] = test_tkk['text'].str.replace('—', "")
test_tkk['text'] = test_tkk['text'].str.replace('«', "")
test_tkk['text'] = test_tkk['text'].str.replace('»', "")

In [170]:
#банки
train['text'] = train['text'].str.replace('’', "")
train['text'] = train['text'].str.replace('”', "")
train['text'] = train['text'].str.replace('“', "")
train['text'] = train['text'].str.replace('—', "")
train['text'] = train['text'].str.replace('«', "")
train['text'] = train['text'].str.replace('»', "")

test['text'] = test['text'].str.replace('’', "")
test['text'] = test['text'].str.replace('”', "")
test['text'] = test['text'].str.replace('“', "")
test['text'] = test['text'].str.replace('—', "")
test['text'] = test['text'].str.replace('«', "")
test['text'] = test['text'].str.replace('»', "")

In [171]:
train_tkk["text"]

0       user максим вашем письмо мы получили наши сотр...
1               мегафон стал владельцем 50 акций евросети
2       rt user user мтс россия прислала жителям херсо...
3                 видео url реклама со смехом мтс супер 0
4       user потому что мтс достало а пчел ненавижу с ...
                              ...                        
9204    понедвечер сейчас буду делать уроки вот такая ...
9205    ну еслиже это мтсто скорость интернета будет г...
9206    rt user у мтс проблемы со связью изза замены о...
9207    rt user у мтс проблемы со связью изза замены о...
9208    rt user у мтс проблемы со связью изза замены о...
Name: text, Length: 9209, dtype: object

In [172]:
train["text"]

0                       url взять кредит тюмень альфа банк
1                      мнение о кредитной карте втб 24 url
2        райффайзенбанк снижение ключевой ставки цб на ...
3        современное состояние кредитного поведения в р...
4                  user user главное чтоб банки сбер и втб
                               ...                        
10720    newzday банк уралсиб повысил ставки по вкладам...
10721    хэйя знаючто ты заказывала одну вещь с офф сай...
10722    это радует  банк уралсиб и фольксваген банк ру...
10723    при финансовой поддержке россельхозбанка постр...
10724    новая машина в кредит с россельхозбанком прост...
Name: text, Length: 10725, dtype: object

Приведение к нормальной форме

In [96]:
!pip install pymorphy2

In [173]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [174]:
twit_norm = lambda x: " ".join([morph.parse(word)[0].normal_form for word in x.split()])

In [175]:
train_tkk["text"] = train_tkk['text'].apply(twit_norm)
train_tkk.head()

,text,answer,company,sample_id
0,user максим ваш письмо мы получить наш сотрудн...,0,beeline,1
1,мегафон стать владелец 50 акция евросеть,0,megafon,2
2,rt user user мтс россия прислать житель херсон...,-1,mts,3
3,видео url реклама с смех мтс супер 0,1,mts,4
4,user потому что мтс достать а пчела ненавидеть...,-1,mts,5


In [176]:
test_tkk["text"] = test_tkk['text'].apply(twit_norm)
test_tkk.head()

,text,answer,company,sample_id
0,rt vzglyad по дело о работа мтс в узбекистан с...,-1,mts,1
1,rt user url кредитный карта мегафон банк,0,megafon,2
2,оформить кредитный карта в банк мтс url,0,mts,3
3,как перевести деньга с билайн на кредитный карта,0,beeline,4
4,начальник отдел кредитный контроль оао мтс уса...,0,mts,5


In [177]:
train["text"] = train['text'].apply(twit_norm)
test["text"] = test['text'].apply(twit_norm)

In [178]:
twit_len = lambda x: len([morph.parse(word)[0].normal_form for word in x.split()])
train_tkk["len"] = train_tkk['text'].apply(twit_len)

In [179]:
train_tkk.sort_values(by="len", ascending=False)

,text,answer,company,sample_id,len
3204,но ты не вернуть всё равно да и как то уже быт...,0,beeline,3225,29
7668,rt user саша и коля гулять по крыша у коли быт...,0,beeline,7438,29
7669,rt user саша и коля гулять по крыша у коли быт...,0,mts,7438,29
8740,билайн дать мой номер не понять кто как узнать...,-1,beeline,8414,28
1392,прийти смс про повышение трафик только что вес...,-1,mts,1404,28
...,...,...,...,...,...
4182,прощаймтс,-1,mts,4198,1
6573,билайн,0,beeline,6390,1
6101,user,0,beeline,5956,1
3531,мтс,0,mts,3542,1


In [180]:
test_tkk["len"] = test_tkk['text'].apply(twit_len)
test_tkk.sort_values(by="len", ascending=False)

,text,answer,company,sample_id,len
2045,я так с мтс бороться месяц два звонить они пар...,-1,mts,2353,30
1799,rt user встречаться два друг у ты мтс если бы ...,0,mts,1953,29
2240,у я в деревня опять нет интрнет не деревня даж...,-1,mts,2659,28
1708,user че у вы за страный сервис в лк мало тот ч...,-1,mts,1808,28
1433,rt user user у алёна мегафон и он не работать ...,-1,megafon,1472,28
...,...,...,...,...,...
1875,почему не билайн,0,beeline,2067,3
213,про ростелеком url,0,rostelecom,188,3
2456,караоке от ростелеком,0,rostelecom,2938,3
1095,мтс гавный,-1,mts,1046,2


In [181]:
train["len"] = train['text'].apply(twit_len)
train.sort_values(by="len", ascending=False)

,text,answer,company,sample_id,len
7791,сейчас я по есть и мы пойти в сбербанк если не...,0,sberbank,6651,28
6380,в филиал на ленинский 42 выдавать талончик сра...,-1,sberbank,5264,26
1799,в ночь на среда о введение в отношение россия ...,-1,vtb,1323,26
1800,в ночь на среда о введение в отношение россия ...,-1,bankmoskvy,1323,26
319,а мы как раз сегодня получить от юрист информа...,-1,gazprom,308,26
...,...,...,...,...,...
7840,в сбербанк,0,sberbank,6699,2
10619,user спасибо,1,sberbank,9444,2
8130,ненавидеть сбербанк,-1,sberbank,6988,2
10266,сбербанк,0,sberbank,9107,1


In [182]:
test["len"] = test['text'].apply(twit_len)
test.sort_values(by="len", ascending=False)

,text,answer,company,sample_id,len
1705,я позвонить из альфа банк и я сказать что я би...,0,alfabank,1739,28
1124,user при долгий пытка поддержка они понести аб...,-1,sberbank,1182,26
1389,sberbank cib не на 100 а на 50 бп теперь ждать...,0,sberbank,1481,26
2149,в 147 я позвонить в сбербанкпочий так позднопо...,-1,sberbank,2191,26
1051,иметь счёт в альфабанка часто заходить в моско...,1,alfabank,1098,26
...,...,...,...,...,...
2679,user спасибо,1,alfabank,2723,2
2203,заебал сбербанк,-1,sberbank,2245,2
2210,user спасибо,1,raiffeisen,2252,2
1847,сбербанк говно,-1,sberbank,1882,2


Длина максимального твита 30

In [40]:
!pip install gensim
!pip install --upgrade gensim

     |████████████████████████████████| 24.2MB 1.5MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [183]:
import gensim
model = gensim.models.KeyedVectors.load('/content/drive/My Drive/Colab Notebooks/181/model.model')

In [184]:
model["куку"]

array([-0.00296784,  0.1947115 ,  0.52625775,  0.39905536, -0.27364528,
       -0.01536282,  0.03818725,  0.23093444, -0.0551637 , -0.10007454,
       -0.44373977,  0.20660451,  0.28840637, -0.41925228,  0.14498287,
        0.16983645, -0.04566146,  0.23128168, -0.03944245,  0.21226472,
       -0.41558653,  0.08801638,  0.11307734, -0.06979934, -0.0097214 ,
        0.07263824,  0.05620824, -0.14806645,  0.35194167,  0.172066  ,
       -0.43698296, -0.12684822, -0.21499002, -0.03457339, -0.03549154,
        0.23319997, -0.00290691,  0.08414913,  0.26712644,  0.09097573,
       -0.16478528, -0.11838078,  0.13538484,  0.22087407, -0.14097059,
        0.01925237, -0.01664282, -0.02799122, -0.19373639, -0.0536807 ,
        0.1361174 ,  0.5166878 , -0.06879478, -0.01188826, -0.37798637,
        0.13728738, -0.21550705,  0.3008633 ,  0.15207118, -0.32287186,
       -0.26013038, -0.10156683,  0.31808707, -0.25897062,  0.0171144 ,
        0.2729003 , -0.4328185 ,  0.01800041, -0.28583258, -0.21

In [185]:
#все твиты сделаем размером 30 на 300

def embedding(text):
  k = len([model[word] for word in text.split()])
  return np.concatenate([np.array([model[word] for word in text.split()]), np.zeros((30-k,300))])

In [186]:
twit_emb = lambda x: embedding(x)
X_emb_train_tkk = train_tkk['text'].apply(twit_emb)
X_emb_train_tkk.shape, X_emb_train_tkk[0].shape

((9209,), (30, 300))

In [187]:
X_emb_test_tkk = test_tkk['text'].apply(twit_emb)
X_emb_test_tkk.shape, X_emb_test_tkk[0].shape

((2460,), (30, 300))

In [188]:
X_emb_train_tkk = np.stack(X_emb_train_tkk.values)

In [189]:
X_emb_test_tkk = np.stack(X_emb_test_tkk.values)

In [190]:
#получили матрицу dim=4 для keras 2d

X_emb_train_tkk = X_emb_train_tkk.reshape(X_emb_train_tkk.shape[0],30,300,1)
X_emb_train_tkk.shape

(9209, 30, 300, 1)

In [191]:
#получили матрицу dim=4 для keras 2d

X_emb_test_tkk = X_emb_test_tkk.reshape(X_emb_test_tkk.shape[0],30,300,1)
X_emb_test_tkk.shape

(2460, 30, 300, 1)

In [192]:
#сделаем тоже самое для банков
X_emb_train_bank = train['text'].apply(twit_emb)
X_emb_test_bank = test['text'].apply(twit_emb)

X_emb_train_bank = np.stack(X_emb_train_bank.values)
X_emb_test_bank = np.stack(X_emb_test_bank.values)

X_emb_train_bank = X_emb_train_bank.reshape(X_emb_train_bank.shape[0],30,300,1)
X_emb_test_bank = X_emb_test_bank.reshape(X_emb_test_bank.shape[0],30,300,1)


In [193]:
import keras
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D, BatchNormalization
from keras.models import Sequential

In [194]:
test_tkk['answer'] = test_tkk['answer'].map({-1:0,0:1,1:2})
train_tkk['answer'] = train_tkk['answer'].map({-1:0,0:1,1:2})

In [195]:
test['answer'] = test['answer'].map({-1:0,0:1,1:2})
train['answer'] = train['answer'].map({-1:0,0:1,1:2})

In [196]:
train_tkk['answer']

0       1
1       1
2       0
3       2
4       0
       ..
9204    2
9205    0
9206    0
9207    0
9208    0
Name: answer, Length: 9209, dtype: int64

In [198]:
y_train = keras.utils.to_categorical(train_tkk['answer'], 3)
y_test = keras.utils.to_categorical(test_tkk['answer'], 3)

In [199]:
y_train_bank = keras.utils.to_categorical(train['answer'], 3)
y_test_bank = keras.utils.to_categorical(test['answer'], 3)

In [200]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=(30,300,1), data_format='channels_last', padding = 'same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(1000, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [207]:
#model2
''' В статье написано так: For all datasets we use: rectified linear units, filter
windows (h) of 3, 4, 5 with 100 feature maps each,
dropout rate (p) of 0.5, l2 constraint (s) of 3, and
mini-batch size of 50. These values were chosen
via a grid search on the SST-2 dev set.'''
model2 = Sequential()
model2.add(Conv1D(100, 3, padding='same', strides=1, activation='relu'))
model2.add(MaxPooling1D(pool_size=28, padding='same'))
model2.add(Conv1D(100, 4, padding='same', strides=1, activation='relu'))
model2.add(MaxPooling1D(pool_size=27, padding='same'))
model2.add(Conv1D(100, 5, padding='same', strides=1, activation='relu'))
model2.add(MaxPooling1D(pool_size=26, padding='same'))
model2.add(Flatten())
model2.add(Dropout(0.2))
#model2.add(Dense(1000, activation='relu'))
model2.add(Dense(3, activation='softmax'))

model2.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [202]:
batch_size = 50
epochs = 10
model.fit(X_emb_train_tkk, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_emb_test_tkk, y_test))
score = model.evaluate(X_emb_test_tkk, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
185/185 [==============================] - 197s 1s/step - loss: 0.9458 - accuracy: 0.5732 - val_loss: 0.9605 - val_accuracy: 0.5293
Epoch 2/10
185/185 [==============================] - 197s 1s/step - loss: 0.8268 - accuracy: 0.6381 - val_loss: 0.9335 - val_accuracy: 0.5947
Epoch 3/10
185/185 [==============================] - 196s 1s/step - loss: 0.7724 - accuracy: 0.6599 - val_loss: 0.9128 - val_accuracy: 0.5996
Epoch 4/10
185/185 [==============================] - 197s 1s/step - loss: 0.7331 - accuracy: 0.6850 - val_loss: 0.9057 - val_accuracy: 0.6114
Epoch 5/10
185/185 [==============================] - 196s 1s/step - loss: 0.6847 - accuracy: 0.7039 - val_loss: 0.9487 - val_accuracy: 0.6033
Epoch 6/10
185/185 [==============================] - 195s 1s/step - loss: 0.6434 - accuracy: 0.7231 - val_loss: 0.9161 - val_accuracy: 0.6061
Epoch 7/10
185/185 [==============================] - 195s 1s/step - loss: 0.6130 - accuracy: 0.7353 - val_loss: 0.9016 - val_accuracy: 0.6211

In [205]:
X_emb_train_tkk = X_emb_train_tkk.reshape(X_emb_train_tkk.shape[0],30,300)
X_emb_train_tkk.shape
X_emb_test_tkk = X_emb_test_tkk.reshape(X_emb_test_tkk.shape[0],30,300)
X_emb_test_tkk.shape

(2460, 30, 300)

In [209]:
#model2

batch_size = 70
epochs = 10
model2.fit(X_emb_train_tkk, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_emb_test_tkk, y_test))
score = model2.evaluate(X_emb_test_tkk, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
132/132 [==============================] - 4s 34ms/step - loss: 0.0868 - accuracy: 0.9705 - val_loss: 1.6337 - val_accuracy: 0.6776
Epoch 2/10
132/132 [==============================] - 4s 33ms/step - loss: 0.0737 - accuracy: 0.9730 - val_loss: 1.6330 - val_accuracy: 0.6415
Epoch 3/10
132/132 [==============================] - 4s 33ms/step - loss: 0.0671 - accuracy: 0.9737 - val_loss: 1.8882 - val_accuracy: 0.6524
Epoch 4/10
132/132 [==============================] - 4s 33ms/step - loss: 0.0575 - accuracy: 0.9755 - val_loss: 2.0443 - val_accuracy: 0.6370
Epoch 5/10
132/132 [==============================] - 4s 33ms/step - loss: 0.0527 - accuracy: 0.9786 - val_loss: 2.0514 - val_accuracy: 0.6398
Epoch 6/10
132/132 [==============================] - 4s 33ms/step - loss: 0.0516 - accuracy: 0.9788 - val_loss: 1.7919 - val_accuracy: 0.6317
Epoch 7/10
132/132 [==============================] - 4s 33ms/step - loss: 0.0647 - accuracy: 0.9698 - val_loss: 2.2113 - val_accuracy: 0.6606

In [203]:
yhat_classes_m1 = model.predict_classes(X_emb_test_tkk, verbose=0)
yhat_classes_m1.shape, test_tkk['answer'].shape

((2460,), (2460,))

In [52]:
yhat_classes_m1

array([1, 1, 1, ..., 0, 1, 0])

In [204]:
f1_macro = f1_score(test_tkk['answer'], yhat_classes_m1, average="macro")
f1_micro = f1_score(test_tkk['answer'], yhat_classes_m1, average="micro")
f1_macro, f1_micro

(0.48826267900881043, 0.6146341463414634)

In [210]:
yhat_classes_m2 = model2.predict_classes(X_emb_test_tkk, verbose=0)
yhat_classes_m2.shape, test_tkk['answer'].shape

((2460,), (2460,))

In [211]:
f1_macro = f1_score(test_tkk['answer'], yhat_classes_m2, average="macro")
f1_micro = f1_score(test_tkk['answer'], yhat_classes_m2, average="micro")
f1_macro, f1_micro

(0.5692380715369485, 0.6695121951219513)

Я попробовала модель 2D без изменений - она дает значения метрик: (0.48826267900881043, 0.6146341463414634)
Я пробовала менять в ней значения kernel size на (3,30), но это не дало прироста метрики и скорости.

Попробовала модель со слоями 1D - она быстрее и метрика чуть лучше получилась. 